<a href="https://colab.research.google.com/github/elena-ln/portfolio/blob/master/Spark_Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Spark Setup 

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://securi

In [ ]:
!wget -q http://apache.forsale.plus/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!ls
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

links.csv    sample_data		      spark-3.0.1-bin-hadoop2.7.tgz.2
movies.csv   spark-3.0.1-bin-hadoop2.7	      tags.csv
ratings.csv  spark-3.0.1-bin-hadoop2.7.tgz
README.txt   spark-3.0.1-bin-hadoop2.7.tgz.1


In [ ]:
!wget -q http://apache.mirrors.pair.com/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz


In [ ]:
# Set up Spark
!pip install -q findspark
!pip install py4j

!export JAVA_HOME=$(/usr/lib/jvm/java-8-openjdk-amd64 -v 1.8)
! echo $JAVA_HOME
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
import findspark
findspark.init("spark-3.0.1-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

/bin/bash: /usr/lib/jvm/java-8-openjdk-amd64: Is a directory
/usr/lib/jvm/java-8-openjdk-amd64


In [ ]:
spark.version

'3.0.1'

### Load package

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [ ]:
import os
os.environ["PYSPARK_PYTHON"]="python3"

### Read data


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving README.txt to README.txt
Saving links.csv to links.csv
Saving movies.csv to movies.csv
Saving tags.csv to tags.csv
Saving ratings.csv to ratings.csv
User uploaded file "README.txt" with length 8342 bytes
User uploaded file "links.csv" with length 197979 bytes
User uploaded file "movies.csv" with length 494431 bytes
User uploaded file "tags.csv" with length 118660 bytes
User uploaded file "ratings.csv" with length 2483723 bytes


In [ ]:
!ls 

links.csv    README.txt			spark-3.0.1-bin-hadoop2.7.tgz
movies.csv   sample_data		spark-3.0.1-bin-hadoop2.7.tgz.1
ratings.csv  spark-3.0.1-bin-hadoop2.7	tags.csv


### Spark read data from drive

In [ ]:
links_df = spark.read.csv('./links.csv',inferSchema=True, header =True)
ratings_df = spark.read.csv('./ratings.csv',inferSchema=True, header =True)
movies_df = spark.read.csv('./movies.csv',inferSchema=True, header =True)
tags_df = spark.read.csv('./tags.csv',inferSchema=True, header =True)

In [284]:
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [ ]:
tags_df.show(5)

+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|     2|  60756|          funny|1445714994|
|     2|  60756|Highly quotable|1445714996|
|     2|  60756|   will ferrell|1445714992|
|     2|  89774|   Boxing story|1445715207|
|     2|  89774|            MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows



In [ ]:
links_df.show(5)

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
|      5|113041| 11862|
+-------+------+------+
only showing top 5 rows



In [ ]:
tmp1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))


For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1


In [ ]:
tmp1 = sum(ratings_df.groupBy('movieId').count().toPandas()['count']==1)
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1,tmp2))

3446 out of 9724 movies are rated by only one user


### Part 1: Spark SQL and OLAP

In [128]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

### 1: The Number of Users


In [ ]:
output = spark.sql("select count(distinct userId) as number_of_users from ratings").toPandas()
output

,number_of_users
0,610


### 2: The Number of Movies


In [285]:
output = spark.sql("select count(distinct movieId) as number_of_movies from movies").toPandas()
output

,number_of_movies
0,9742


### 3: How many movies are rated by users? List movies not rated before




In [ ]:
output_1 = spark.sql("select count(distinct movieId) as number_of_ratedMovies from ratings").toPandas()
output_2 = spark.sql("select m.title, m.genres from movies m left join ratings r on m.movieId = r.movieId where r.userId is null").toPandas()

In [ ]:
output_1

,number_of_ratedMovies
0,9724


In [ ]:
output_2

,title,genres
0,"Innocents, The (1961)",Drama|Horror|Thriller
1,Niagara (1953),Drama|Thriller
2,For All Mankind (1989),Documentary
3,"Color of Paradise, The (Rang-e khoda) (1999)",Drama
4,I Know Where I'm Going! (1945),Drama|Romance|War
5,"Chosen, The (1981)",Drama
6,"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance
7,Scrooge (1970),Drama|Fantasy|Musical
8,Proof (1991),Comedy|Drama|Romance
9,"Parallax View, The (1974)",Thriller


### 4: List Movie Genres

In [105]:
output = spark.sql("select distinct genres from movies").toPandas()
output

,genres
0,Comedy|Horror|Thriller
1,Adventure|Sci-Fi|Thriller
2,Action|Adventure|Drama|Fantasy
3,Action|Drama|Horror
4,Action|Animation|Comedy|Sci-Fi
...,...
946,Action|Adventure|Animation|Comedy|Thriller
947,Comedy|Drama
948,Drama|Mystery|Romance
949,Action|Comedy|Drama|Thriller


In [107]:
from pyspark.sql.functions import split, regexp_extract

In [112]:
from pyspark.sql.types import *
from pyspark.sql import SQLContext


In [185]:
one_genre = spark.sql("select movieId, title, explode(split(genres, '[|]')) as genre from movies")

In [186]:
one_genre.show()

+-------+--------------------+---------+
|movieId|               title|    genre|
+-------+--------------------+---------+
|      1|    Toy Story (1995)|Adventure|
|      1|    Toy Story (1995)|Animation|
|      1|    Toy Story (1995)| Children|
|      1|    Toy Story (1995)|   Comedy|
|      1|    Toy Story (1995)|  Fantasy|
|      2|      Jumanji (1995)|Adventure|
|      2|      Jumanji (1995)| Children|
|      2|      Jumanji (1995)|  Fantasy|
|      3|Grumpier Old Men ...|   Comedy|
|      3|Grumpier Old Men ...|  Romance|
|      4|Waiting to Exhale...|   Comedy|
|      4|Waiting to Exhale...|    Drama|
|      4|Waiting to Exhale...|  Romance|
|      5|Father of the Bri...|   Comedy|
|      6|         Heat (1995)|   Action|
|      6|         Heat (1995)|    Crime|
|      6|         Heat (1995)| Thriller|
|      7|      Sabrina (1995)|   Comedy|
|      7|      Sabrina (1995)|  Romance|
|      8| Tom and Huck (1995)|Adventure|
+-------+--------------------+---------+
only showing top

In [151]:
output =  spark.sql("select distinct explode(split(genres, '[|]')) as one_genre from movies").toPandas()
output 

,one_genre
0,Crime
1,Romance
2,Thriller
3,Adventure
4,Drama
5,War
6,Documentary
7,Fantasy
8,Mystery
9,Musical


### 5: Movie for Each Category

In [187]:
output= one_genre.groupBy("genre").count().toPandas()

In [188]:
output.sort_values('count',ascending=False)

,genre,count
4,Drama,4361
16,Comedy,3756
2,Thriller,1894
18,Action,1828
1,Romance,1596
3,Adventure,1263
0,Crime,1199
19,Sci-Fi,980
14,Horror,978
7,Fantasy,779


In [198]:
from pyspark.sql.functions import concat_ws, collect_list

In [199]:
output_3 = one_genre.groupBy("genre").agg(concat_ws(",", collect_list("title")).alias("list_of_movies"))

In [203]:
output_3.toPandas()

,genre,list_of_movies
0,Crime,"Heat (1995),Casino (1995),Money Train (1995),G..."
1,Romance,"Grumpier Old Men (1995),Waiting to Exhale (199..."
2,Thriller,"Heat (1995),GoldenEye (1995),Money Train (1995..."
3,Adventure,"Toy Story (1995),Jumanji (1995),Tom and Huck (..."
4,Drama,"Waiting to Exhale (1995),American President, T..."
5,War,"Richard III (1995),Misérables, Les (1995),Brav..."
6,Documentary,"Nico Icon (1995),Heidi Fleiss: Hollywood Madam..."
7,Fantasy,"Toy Story (1995),Jumanji (1995),City of Lost C..."
8,Mystery,"Copycat (1995),City of Lost Children, The (Cit..."
9,Musical,"Pocahontas (1995),Muppet Treasure Island (1996..."


### Part 2: Spark ALS based approach for training model
###### will use an Spark ML to predict the ratings. First, let's reload "ratings.csv" using sc.textFile and then convert it to the form of (user, item, rating) tuples.

In [204]:
ratings_df.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [205]:
movie_ratings = ratings_df.drop("timestamp")

In [206]:
# Data type convert
from pyspark.sql.types import IntegerType, FloatType
movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))

In [207]:
movie_ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



### ALS Model Selection and Evaluation
With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [225]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit

In [209]:
#Create test and train set
(training,test)=movie_ratings.randomSplit([0.8,0.2])

In [261]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# Create ALS model
als = ALS(maxIter=5, rank=10, regParam=0.01, userCol="userId", itemCol="movieId",ratingCol="rating",
          coldStartStrategy="drop", nonnegative = True)


In [262]:
# Tune model using ParamGridBuilder
param_grid = ParamGridBuilder()\
            .addGrid(als.rank, [12,13,14])\
            .addGrid(als.maxIter,[18,19,20])\
            .addGrid(als.regParam,[.17,.18,.19])\
            .build()

In [263]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [264]:
# Build cross validation using TranValidationSplit
tvs = TrainValidationSplit(
    estimator = als,
    estimatorParamMaps = param_grid,
    evaluator = evaluator
)

In [265]:
#Fit ALS model to training data
model = tvs.fit(training)


In [266]:
# Extract best model from the tuning exercise using ParamGridBuilder
best_model = model.bestModel


In [267]:
#Generate predictions and evaluate using RMSE
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)


In [286]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank:", best_model.rank)
print (" MaxIter:", best_model._java_obj.parent().getMaxIter())
print (" RegParam:", best_model._java_obj.parent().getRegParam())

RMSE = 0.7075160370245244
**Best Model**
 Rank: 12
 MaxIter: 20
 RegParam: 0.17


In [287]:
predictions.sort("userId","rating").show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|   1445|   3.0|  2.596446|
|     1|    780|   3.0| 3.8492374|
|     1|   2414|   3.0| 3.8647115|
|     1|    673|   3.0| 3.1803024|
|     1|   2528|   3.0| 3.5904841|
|     1|   1060|   4.0| 4.1096606|
|     1|   1793|   4.0| 3.7091136|
|     1|    441|   4.0|  4.617717|
|     1|   2000|   4.0| 4.3694077|
|     1|      6|   4.0|  4.376753|
|     1|   2985|   4.0| 3.9490318|
|     1|   3440|   4.0| 2.5169213|
|     1|   2174|   4.0| 4.2218595|
|     1|    733|   4.0|  4.214119|
|     1|   2492|   4.0|  2.726708|
|     1|   1552|   4.0| 3.2946324|
|     1|   2366|   4.0| 4.1075363|
|     1|      3|   4.0| 3.6382527|
|     1|    235|   4.0|  3.897194|
|     1|   3703|   5.0|  4.425583|
+------+-------+------+----------+
only showing top 20 rows



### Model Apply and see the performance

In [288]:
alldata=best_model.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.7075160370245244


### Recommendation Application

In [289]:
# Generate top 10 movie recommendations for each user
userRecs = best_model.recommendForAllUsers(10)
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[3379, 4.980732]...|
|   463|[[3379, 5.0963793...|
|   496|[[3379, 4.782264]...|
|   148|[[67618, 4.539251...|
|   540|[[3379, 5.456634]...|
|   392|[[3379, 4.8132024...|
|   243|[[67618, 5.756536...|
|    31|[[3379, 5.367116]...|
|   516|[[3379, 5.0176477...|
|   580|[[60943, 4.90546]...|
|   251|[[3379, 5.8792315...|
|   451|[[3379, 5.271068]...|
|    85|[[7121, 4.9022923...|
|   137|[[3379, 5.160457]...|
|    65|[[3379, 5.016526]...|
|   458|[[67618, 5.648513...|
|   481|[[3379, 4.0065484...|
|    53|[[3379, 7.0868354...|
|   255|[[7842, 4.2383676...|
|   588|[[3379, 4.5309005...|
+------+--------------------+
only showing top 20 rows



In [273]:
# Generate top 10 user recommendations for each movie
movieRecs = best_model.recommendForAllItems(10)
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|   1580|[[53, 4.7752595],...|
|   4900|[[53, 4.5284266],...|
|   5300|[[53, 4.42143], [...|
|   6620|[[53, 4.718343], ...|
|   7340|[[53, 3.9199052],...|
|  32460|[[53, 5.4944806],...|
|  54190|[[53, 5.4139543],...|
|    471|[[53, 4.4320016],...|
|   1591|[[53, 3.6215606],...|
| 140541|[[544, 4.2444463]...|
|   1342|[[53, 3.2765746],...|
|   2122|[[53, 3.8431854],...|
|   2142|[[43, 3.8397465],...|
|   7982|[[547, 4.969271],...|
|  44022|[[53, 4.6301527],...|
| 141422|[[53, 3.1845493],...|
| 144522|[[53, 3.1497045],...|
|    833|[[43, 3.645743], ...|
|   5803|[[53, 3.6987267],...|
|   7833|[[53, 4.3765216],...|
+-------+--------------------+
only showing top 20 rows



In [351]:
def get_recs_for_user(recs):
  #Recs sould be for a specific user
  recs = recs.select("recommendations.movieId","recommendations.rating")
  movies = recs.select("movieId").toPandas().iloc[0,0]
  ratings = recs.select("rating").toPandas().iloc[0,0]
  ratings_matrix = pd.DataFrame(movies, columns=['movieId'])
  ratings_matrix["ratings"]= ratings
  ratings_matrix_ps = SQLContext.createDataFrame(ratings_matrix)
  return ratings_matrix_ps

Recommend movies to users with id 575, 232




In [360]:

users = ratings_df.select(best_model.getUserCol()).filter("userId == 575 or userId=232").distinct()
userSubsetRecs = best_model.recommendForUserSubset(users,10)
userSubsetRecs.show()



+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   232|[[3379, 4.765686]...|
|   575|[[3379, 4.9201865...|
+------+--------------------+



Recommend simiar movies for movie with with id 463, 471


In [361]:
movies = ratings_df.select(best_model.getItemCol()).filter('movieId==471').distinct()
movieSubSetRecs = best_model.recommendForItemSubset(movies, 10)
movieSubSetRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|    471|[[53, 4.4320016],...|
+-------+--------------------+

